In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mtick
import random
from math import sin, cos, sqrt, atan2, radians
from ease_grid import EASE2_grid
grid_size = 36000
egrid = EASE2_grid(grid_size)
assert egrid.shape == (406, 964)
from impacts import IMPAaCS

In [2]:
# approximate radius of earth in km
def distance(lat1,lat2,lon1,lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return(distance) #km

In [5]:
I = IMPAaCS(egrid)
for t in range(452,500,1):
    state_file = './impact_states/impacts_E1_t{:d}.pkl'.format(t)
    with open(state_file, 'rb') as fb:
        impact_states = pkl.load(fb)
    impact_states[list(impact_states.keys())[0]][:] = 35
    impact_states[list(impact_states.keys())[1]][:] = 70
    I.grid_cell_state = impact_states
    I.sim_time=t*1000000
#    I.plot_map(save_figure=True, plot_figure=True)    
    I.plot_map_and_bar(save_figure=True, plot_figure=False, fig_path = "./figures/directory/")


In [ ]:
I = IMPAaCS(egrid)
for t in range(0, 310, 25):
    state_file = './impact_states/impacts_E1_t{:d}.pkl'.format(t)
    with open(state_file, 'rb') as fb:
        impact_states = pkl.load(fb)
#    # To force the scale to be consistent
    impact_states[list(impact_states.keys())[0]][:] = 35
    impact_states[list(impact_states.keys())[1]][:] = 70
    I.grid_cell_state = impact_states
    I.sim_time=t*1000000
    I.plot_map(save_figure=False, plot_figure=True, fig_path = "./figures/directory/")

In [ ]:
def plot_map_and_bar(impact_states, t, save_figure=False, plot_figure=False):
    if not plot_figure and not save_figure:
        print('not plotting figure')
        return
    z = np.zeros([self.n_x, self.n_y])
    bar_list = []
    for i, ilon in enumerate(self.egrid.londim):
        for j, ilat in enumerate(self.egrid.latdim):
            grid_cell = str(round(ilon,4))+' '+str(round(ilat,4))
            temp_state = np.mean(self.grid_cell_state[grid_cell][0:2])
            temp_state = re_bin_sio2(temp_state)
            z[i, j] = temp_state
    
        # Don't analyze anything close to polar
        if np.abs(float(i.split(" ")[1])) > 45:
            continue
        else:
            mean_sio2 = np.mean(impact_states[i][0:5])
            bar_list.append(re_bin_sio2(mean_sio2))

    X, Y = np.meshgrid(self.egrid.londim, self.egrid.latdim)
    bar_data = {}
    for u in np.unique(total_list):
        bar_data[u] = bar_list.count(u)/len(total_list)
    
    fig, ax1 = plt.subplots(1, 2, figsize=(12, 5))

    #ax1.contourf(X, Y, np.transpose(z))

    cs = ax1.contourf(X, Y, np.transpose(z), vmin=34, vmax=70) 
    cbar = fig.colorbar(cs, ticks=range(34,70,2))

    ax1.plot(1, 1)
    ax1.set_title('top 2-meter sio2 at time {}myr.png'.format(int(self.sim_time/1000000)))
    ax1.set_xlabel('longitude')
    ax1.set_ylabel('latitude')
    ax1.set_xlim([-90, 90])
    ax1.set_ylim([-45, 45])
    ax2.plot(1, 2)
    ax2.bar(list(bar_data.keys()), list(bar_data.values()))
    ax2.xlim([35,70])
    ax2.ylim([0,.7])   
    
    if save_figure:
        plt.savefig('./figs/maps/sio2_map_at_time_{}myr.png'.format(int(self.sim_time/1000000)), 
                bbox_inches='tight')
    if plot_figure:
        plt.show()
    plt.close()

In [ ]:
I = IMPAaCS(egrid)
for t in range(0, 50, 25):
    state_file = './impact_states/impacts_10min_45lat_t{:d}.pkl'.format(t)
    with open(state_file, 'rb') as fb:
        impact_states = pkl.load(fb)
#    # To force the scale to be consistent
    impact_states[list(impact_states.keys())[0]][:] = 35
    impact_states[list(impact_states.keys())[1]][:] = 70
    I.grid_cell_state = impact_states
    I.sim_time=t*1000000
    I.plot_map(save_figure=False, plot_figure=True)

In [ ]:
min_value = 100

for t in [5, 10, 15, 20]:
    state_file = './impact_states/impacts_10min_45lat_t{:d}.pkl'.format(t)
    print(state_file)
    with open(state_file, 'rb') as fb:
        impact_states = pkl.load(fb)
    total_list = []
    for i in impact_states.keys():
        
        # Don't analyze anything close to polar
        if np.abs(float(i.split(" ")[1])) > 45:
            continue
            
        mean_sio2 = np.mean(impact_states[i][0:5])
        total_list.append(re_bin_sio2(mean_sio2))
        
        if np.min(impact_states[i][:]) < min_value:
            min_value = np.min(impact_states[i][:])
            min_state = i
        
    print('max', np.round(np.max(total_list), 2))
    print('mean', np.round(np.mean(total_list), 2))
    print('median', np.round(np.median(total_list), 2))
    print('min', np.round(np.min(total_list), 2))
    
    bar_data = {}
    for u in np.unique(total_list):
        bar_data[u] = total_list.count(u)/len(total_list)
    
    plt.bar(list(bar_data.keys()), list(bar_data.values()))
    plt.xlim([35,70])
    plt.ylim([0,.7])
    plt.show()
    plt.close()


In [ ]:
min_value = 100

for t in range(0, 310, 25):
    state_file = './impact_states/E1/impacts_E1_t{:d}.pkl'.format(t)
    print(state_file)
    with open(state_file, 'rb') as fb:
        impact_states = pkl.load(fb)
    bar_list = []
    for i in impact_states.keys():
        
        # Don't analyze anything close to polar
        if np.abs(float(i.split(" ")[1])) > 45:
            continue
            
        mean_sio2 = np.mean(impact_states[i][0:5])
        total_list.append(re_bin_sio2(mean_sio2))
        
        if np.min(impact_states[i][:]) < min_value:
            min_value = np.min(impact_states[i][:])
            min_state = i
        
    print('max', np.round(np.max(total_list), 2))
    print('mean', np.round(np.mean(total_list), 2))
    print('median', np.round(np.median(total_list), 2))
    print('min', np.round(np.min(total_list), 2))
    
    bar_data = {}
    for u in np.unique(total_list):
        bar_data[u] = total_list.count(u)/len(total_list)
    
    plt.bar(list(bar_data.keys()), list(bar_data.values()))
    plt.xlim([35,70])
    plt.ylim([0,.7])
    plt.show()
    plt.close()
